<h1>Job Posting Finder</h1>
<h3>Goals for this project:</h3>
<ul>
<li>Use Selenium to scrape www.indeed.com for attractive Data Scientist positions.</li>
<li>Collect the results in a Pandas DataFrame.</li>
</ul>

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

<h1>Get the Data</h1>
<ul><li>To get started, I'll go ahead and search "data science" on indeed.com.</li>
    <li>The subsequent pages of search results are treated as separate URLs, which don't seem to have a pattern, so I need to hard-code them in:</li></ul>

In [2]:
url_list = [
    "https://www.indeed.com/jobs?q=data+science&l=&from=searchOnHP&vjk=53bfde300305e5db",
    "https://www.indeed.com/jobs?q=data+science&start=10&pp=gQAPAAABhdE7NrAAAAAB9eqlhwAwAQAA80PNlh5uqedI-H_Z7TWYMx2Mdz3DNe04MhmHm7j22P4Wj4QJaBOWVO6bxIkmAAA&vjk=b2798c454ed85222",
    "https://www.indeed.com/jobs?q=data+science&start=20&pp=gQAeAAABhdE7NrAAAAAB9eqlhwBOAQIBCSQHA_mPl9jNvVNgW5yWUNyAJGSPmmFLBFvKn8GSVGCrXI72pEhclZbkVjrgbHTy-BwhQd6vYnFPcmVZXi1lhWz6IsHCYukiDsxEAAA&vjk=0819737b36346a92",
    "https://www.indeed.com/jobs?q=data+science&start=30&pp=gQAtAAABhdE7NrAAAAAB9eqlhwBjAQIBCSIKBWWrTUnY0sHkiBMYtMx7IqAUwjqfOEfWx1adFnjl1s4yklKXCME6T_hiAfWkXqEQ3gyo0LIOkOXvTSYNKAaiD16_J5ztakIOJ5xXtspNj0wVUfdsPEUUSVS-cH7lAAA&vjk=9b709d0ed5fc5a55",
    "https://www.indeed.com/jobs?q=data+science&start=40&pp=gQA8AAABhdE7NrAAAAAB9eqlhwB4AQIBECIIQBNjpbifmCLkkTiaCLRdbL-IOgtfrea3rXKUGpIsfRGFNIvEkRHsAEKd_ssr-72tEHm4-fySwgYA63i_WqCVHtSVukqvFxLdBixrYRo3D3gvpyFkZv57Jf8Znx0hwdEsGdiIDuCMAqVLI8m7EQbjOiLsAAA",
    "https://www.indeed.com/jobs?q=data+science&start=50&pp=gQBLAAABhdE7NrAAAAAB9eqlhwCLAQIBECIOAPtHL-fw2KIv02786NWAdeBoTEAqiw1RJBUTV857PdaZGJAg2R5BF07kFACG6z-FgsCGDm7uozn8ETg4YuXr6Co_OA0Yxnv9vLkcK82LtcYUw25ogNYTmg2vzfM3fvEh9dUQ21MyPfk5trjJTNNE14HvUUiu5F6VFVDfMiiCcH4hFHc0ggAA",
    "https://www.indeed.com/jobs?q=data+science&start=60&pp=gQBaAAABhdE7NrAAAAAB9eqlhwCgAQIBECIMCWXafFwFigj-yFE-Csq4FA1DsXKK49QfG2yQ2LDhK0wMO7GYuD0oXiOqXjne_uj3Q_9gyp26lbssoltCSknDbXYdZwgHxsVRwcPA6BkX17GYlzgfNgYEFi9E7LM7WkQu72u3_WEnTZO_cJjy9QniA8f9QBq_l6MGTwXveYeVg3_Gz-KzAp-NIh2KXrvPwRonh9pPMa3NdzL3FAAA",
    "https://www.indeed.com/jobs?q=data+science&start=70&pp=gQBpAAABhdE7NrAAAAAB9eqlhwCxAQMBECIMNgcCNKkTbSgMD1ab-zKRy0OIawxABQTcqDDu_vuTLhgXWG3nxERZRsUESH5k8CKp-QS6favOqFj_RJ06jDdXycRdmvNwY0CIJiDd5cN35rpoqAskZibGgqHldz3GmFbRtlUeT2OIwoU0938sksCgxT43we66XnjgM8KNApR8T0sJwgEGAsUpOL6kUY4jv7Goee9v0P7xxxvwswtmCNBtonvi2Vgt0_3SjkDBAAA",
    "https://www.indeed.com/jobs?q=data+science&start=80&pp=gQB4AAABhdE7NrAAAAAB9eqlhwDJAQMBECIMNgcBB3Kw56foPvALz8_gcEZF3k5yVKaIjALC4ONSiHtxlEbwKZaY2zQG8pdyKgi7W3vTMRLQJYNpCP_4d0JWzELpjGd6n1elQ-Fyy0EWSqTpDxCYDJWxJDjs0LfJd8nQCffCFzOo9I0yIozU-IxZAKFXaMhBs-igihNcsytGLZ40IsyUl4vwQsZL2XuTxYJYjGbeeFFGRTnOhPOSlkrK3cqc3ZKDN1_2gc_Xo5wmbgp2qx-eceBO_W32ZYPVSqF03Mp_AAA",
    "https://www.indeed.com/jobs?q=data+science&start=90&pp=gQCHAAABhdE7NrAAAAAB9eqlhwDVAQMBECIMLAwE7AiRhKU9SXGhO-Oxw4_7qdQNPdYx1P3RDwGCvHaJ_52dJZIRetFmF5a9EzR1xgkyB-_BGfoj5fV0osQsoggtaFx6REHveJfTqR3yIJza7Qb9mV7BmiYqZrwGy7Wn3BsYWR5vGsrUgd2AduS6YkATa2APjFzAWYMcf54USRfS_Xag7kJNGo4BCeh8yU7n9yKxUNlcvAbosXUm7SbY-s79yrvNkPGkImDFarwnI1-OaBJVSIMMcHj__vtQRjyTP_zQVmtBZV0nXxn17aMKAAA",
    "https://www.indeed.com/jobs?q=data+science&start=100&pp=gQCWAAABhdE7NrAAAAAB9eqlhwDlAQQBECIQLAwaBwDUivOICXteSm1f0mCeVjIyb_0XRVr3lmLPPeM59HtQMBVrIg5lrSqGi3b_2WmUIM-GUL6kZ9GSzyAuo3kIKEoZixtnqWSZoKrujKvg7ESgy_syCqvE3Na0VeCJKwVXj_8upPU2GMMLstn7V18HT-n3UBYseIaC0yl48cD1jKL4oKKbGKF4qPZEXCq3fD5bhl2m1-iLkPyXV-pkfAouxdrKSaZsnrgOAdIksgWI0iwlIDCVoZGrYXP1ke0lk-LuLVpzXVqIB7m9Ng_3NjfTvQOC2IzThp-nWpoZPQAA&vjk=922fab2a3d51bc65",
    "https://www.indeed.com/jobs?q=data+science&start=110&pp=gQClAAABhdE7NrAAAAAB9eqlhwDzAQMBIVAMGgoBethoyMIQckJ3WDdIht2SWLlHPf_7k81A66iv-90H52Ln1shwy4dc_hqzopEXVzBFGwGKYnPuBVjVpd8aKOlszYQacEJI5Fu2hWTQ8aJWoVxlWOd8vFoX7aG2y0FRGZWpcxaA8FTBNcVaRy22mXyFw12Q90QIN2XJ04mygzOM3hVUlw96UeQa8Wcg8FrDZ4_qj_4kKi_CKm_kPOJpmeJlHZiyGU7swC77W_mTj8qyNv8gr3RFAwbNdy8F-LGrsKqQrfGaXDkSHu5u1H1-n6X7jrByC2G5s-mZiIqTUzJvGtwEi-xvaUNSqtLJAAA",
    "https://www.indeed.com/jobs?q=data+science&start=120&pp=gQC0AAABhdE7NrAAAAAB9eqlhwD9AQQBIVAXWAhEBw9ejWM89R1n4J97iOElnZ7U4qT5zOYpQ3Nr0Ejhc2arMP1ZBeH70yBDZmh0Sz-nYRMMcMqxnQicIWwESWCQvM4f9t0ZDO-MBez1_X6EX_FxCpf7IZnhFb26Uc1Ror5ceqcH1hutw4iCjDbg4FUQHRv6_07ucgee-hy8gWF9VNOliWokG8gtF0QDHEJISmyaPKqqmB6I6oTnJKpJ5-TEIhKNLh9sD3d2qA9dKM0i3L-XsUNQ8OXZ4ZqduOs2PqO1F6dIH1Cv2WfJulhFb1wLaxTKx4O2I4JKsIjAuPrwRjBZXLINN5ZADXwl3QXdYT8PJA-MWQAA",
    "https://www.indeed.com/jobs?q=data+science&start=130&pp=gQDDAAABhdE7NrAAAAAB9eqlhwEGAQYBIVAZPAgcCC4GFgYJev1gBhHKrrseney5JGysqWI15dYPGNAtdp26WmaoYkxFp7dW4NEoIDEi0dKPkHQqpkLwU_sqoYAWIkYDhPKcuttIKoE_3m5e7IzXZPhQaq1Y6ub37XT5DlEb6709x_tLdJ4rl6_z-vT3KksSiffcsgzveO3h5YKInyZFUmGKFn5BHGO0iNggFu-y0f0joklGYUK6XC3-wEb1ic7Lk540qlQzdyUHuWJQU13ZuRYU9J5A35-1SGkLi8yqQP0KXaNL0Iz0xaULNZU4RFEfnA8ljftNj4cSBIYqdeKP14agCioI7d87KG4t32V3ElMiSRyuqZqtqzsS7gAA",
    "https://www.indeed.com/jobs?q=data+science&start=140&pp=gQDSAAABhdE7NrAAAAAB9eqlhwESAQcBIUYeSAceCCoJGgZOBgObRQTAdwwCy9YC1567s5v_drqubMnY-og8bS0dofe54fJ8odIuNzVXolEHeBWKkVWwF0xrW7CX7SR-zLz-5rBXwjCN5OczQMzUhauzpOeKOzPOhAYv-pfuUBf6ymXwlBvCLdwWogCFrh305ZJqrGVkSe5AC383HqtdJgmVCnuLd_s-bCLOrGpwqimuMHAU7OtLHn1r_QmjnG9E94Wqt2uW7bz2xW2w-GcvQW1zc2hCPE6xmVqnHxv4o5HnRfS5RVWVaziD6DW4ghE0SYnUt72_m7iF1KOdJxijRwAPtyPl1AkGgZ_kkSTy-Tezp_9Y7tgSV1JY_0ysdq0gANCQ5z2swwAA",
    "https://www.indeed.com/jobs?q=data+science&start=150&pp=gQDhAAABhdE7NrAAAAAB9eqlhwEdAQcBIUYhSAceCCoJGgZSBgHWjmqtyM5d9O5cWy8_NaYoZr_Id_pkMoAl9Gaj2YlQ-b0TnktF00IQrZnlq37XoiKgpFJOJnXKUD8RV5ELIqLK7R_meG1dS_zj9bnSOeoEJMchLvWAIJWO5ZHyDa9EUWhweNvt4qNfHg41Fycr7bebcANBS5G36tfbeoBl6mL6yrwDVSnj1MbuEU9Lb4mCEPoms--C_Ut3IVmx4SI8-KKtNMlVzsQkt0Dzt0fl5Z4dMl9r35UBcuDgfdYPeOogrqiN6B04SpIDym4SfkU9_vMI_7PZakrTHGptOk-eA8hH0J20Z7vr6k4NOmukCnVUcAhp1XBkM2xmrB5O0EOT1EtdK1oSc5dFaQYVvCpIAAA",
    "https://www.indeed.com/jobs?q=data+science&start=160&pp=gQDwAAABhdE7NrAAAAAB9eqlhwEoAQcBIUYhRgogCCgKHAZWBgaPCQ_2iZ18Jqc3UL-bG4actChoH1bvU-EpHzTjaNymbaoJlAfMwdw4r_HFmQAdSaT2A1ofDxQ_O-jxTOUQP_2ZDgtqwxMPeetUtg0PUyl9IPw3mNmTaZdMaXkMa6V1kCZFIhM36zUN43tVM78kCbIY5nmaENNQAu0mpjgHl9dMrei7mrpZX1pAVc9rids25QfGiiGZQcq8fjpAlYkaX1qWOapX7F1ZfWyTC6nzF4A26YdwCguzjtpljTIpoydJqqwSgmDU3jyj4014d_5mVGixCETRv83nIhMTH_Y5BFQzDcPaiCN5sPh6ZvbpzRp5ozNUjSdW2utZmYL_z3XpdtQpil_A97P0Kw3Mcf9gIYqLiDc7Q9juK6AAAA",
    "https://www.indeed.com/jobs?q=data+science&start=170&pp=gQD_AAABhdE7NrAAAAAB9eqlhwEyAQcBIUYhRgogCCgKHAZYBnpIHXORrrwzRMFGTaRiVHBfDCoRHFQ4KlSAPeROjkVjvbb4hfzxq4-hJl1NN8ls8oSvNv-m9HKVwX0B2hk2VDFAUlmmihlMMuIuHQXH-lO8C_8h9wtiLeQilQsVvUhEqDc9Pjza7oQmiqtgf-mn3vp9U_X4EP-R0ddLJ1g0ZDyDh1-ta_gvVkGIVYTO1VHD9KlYHuiZhW0gyevlutsckvUx7r5dzFmkXLFAWeEAgx4_a2n0YcvgVNwOAj86KZyN_lHk2G_ztHDWaSjPZpQjnRFGEfA52bTAieqjHu3oZIv2SEed9uUrHNbcXbHNaQ2QzkzwJLR7PRKBnrB7TzZHUQfz6LNJUkamxdZBEVBO0Vo3O9N3HJAxlmCDKoUb9VbZQNjkAAA",
    "https://www.indeed.com/jobs?q=data+science&start=180&pp=gQEOAAABhdE7NrAAAAAB9eqlhwE7AQgBIkYiSAogCCgKHAZcBlAGA207_FQcrquoJ7pgXo2QzG9QytDmJMfbeF6mBWXqc2nvIaGtNCCU1yqpOobJQ7K__GVs40IaTBgctVY9GxjPtBZjOFLzH1cCVsQPpLiplwAhuT7lxLwmzyx0qZwWFgOtd8hHJoP93rNCXeMIQdkCoc92kqpyB5H8JG2Gji9nU1CTD_8GXR36BBcENFHht810SIg-ey7UK5At4istCXwiNcrgpeNR2ub2chxeMOEYANe7pzp6J8zOG1cFytf80awYXbCeaVzA9AN5itouUo53Yekfj6bNezJm4GgqdsC4sY7wAd7rlKfON1Y8nS5wZY50GDPYfQERZC0uZz1icYkkD2omyiP9ggUeF5tXZ8j30D6gNI-cVi0hrhWOmJ6AxAPXXLsYJyqKNdMmAAA",
    "https://www.indeed.com/jobs?q=data+science&start=190&pp=gQEdAAABhdE7NrAAAAAB9eqlhwFIAQkBIkYiSAogCCgIFglkBg4GQgYBvzwESQT_qjcD7AJIZIFJMaAXDUePwlXjuK0Qi-kkmPapBKSSn0NXmoQs5fgfb5_cRfBI5WzPV0wsUh5np4JFJuy2zZB6oLsNHaIOKGCukZjDXDa18EnuWMV2UD3OJy_ael8_rpA-9nD0NvYFxi6_iK4AVZTUEaPPispuvQ_JIiDDo0YhSg02xKURNQv1XC-mnxlkQv5bMMgmrTm7EtMwk4jCs6XuvJ24Cqe-qwHGRGCCHXr3Snt94FftYzUIW-8USTNg8W0LSjr8GLq8Y3mR-rPtuxttPlJyoKX7Ljcfll9wGYegFM2Uzqysj5riyllWxa1fbpUYI3zqSReFXtjAotyc4jQz2PW6Ox5grIGHW8v6oLZ-mrs6e_3qBtBtdFcKpKb2_Wf_Nw980d2iKLuXWd4dTQAA",
    "https://www.indeed.com/jobs?q=data+science&start=200&pp=gQEsAAABhdE7NrAAAAAB9eqlhwFOAQkBIkYiSAogCCgKFglkBg4GRAYKG2E1AXbjDah8tgrE3Mzy5xONW3jfLZcoo6e54a2yRNtiCWDBxqa6zwh9whYHOIGWmrAMpNBoe1y4vYBVaKu0FzZ-y2u-HwYx4Xrv_VXIGWLRxQfB7b2wnOR1l887EIOCRIj2MgSvv-efIlGhAaEuzXTcOZ3Dn8hDGlEpD5fY6wJLczdPr-8GK_jcUZIoceTA3rS0wsvXu0yRY5mC_yQJ0QLe0nog7Mk1IdyNuCqpoQwifz9xL76-RXFpAY4GvGELBkGv6d77b0wzw4_6oOUKABrSTVnLYHVvoAsX6vMoYYqqRCC_3c9VPiaDeGEfevad8HFgMQf_8vwjfXFY49XDEuvl_sRDzUNNibDBqUpWc6hrqm_zkCw_N4q5TBKoxvJxyw1kg6a0vMBPR1q29-p1lrw2kKUqRHjhawAA",
    "https://www.indeed.com/jobs?q=data+science&start=210&pp=gQE7AAABhdE7NrAAAAAB9eqlhwFRAQkBIkYjSA8gDSgKFglmBxAHRgYZE_O4GOf5ycEOt5mm6IkUXhP34udim6THBlReghBIwQP1bV7b2jVdjEI96DqRMyihENLJKvscAgZ-zMf8WHn-qC7PNc7yK8vq_CJ3pBPP5ATJegsiMiinbAMSNOhmR7U5iC8D-YJ6Rp-k-5_70C-Ly-45Erlgq_gdFdBwKECqakzanncfN93ZZAmgbnWCIi8vo1O4j28APd5AkY9knq5P99SZWpoJR4uQrtGfQwXBx52uisZYiPVGDfQ8x_f608d_MuvcpYd23DHSwCrrLezkk7qpetJweqtZFrXz6SlyVTiHa5kJNBp9uYITigCorb7RzSooSVi7P4fO9yG2dB1ji2t6oSWSuBZdIdXom7k0NQaQOTNKxhM3jPipK1wPS22J6PCNfa5hMRCKVAcbHbc0_4Ja2uxopAPE3XX9xgAA",
    "https://www.indeed.com/jobs?q=data+science&start=220&pp=gQFKAAABhdE7NrAAAAAB9eqlhwFYAQkBIkYjSA8gDSINHAlmBxAHSAYD95TDTqg0l0EqW05D0BJP0CPHDqBBXxd6hmCrSqtwkLiQ8urZwa3F8ZZnDHTx5RzReG6OOSnS2sbN7bCgbeX2D3ro2Bo7M8d4i5ITQiJd38ocmPs05SwVGygpzQHh8Vte5Je2HGrGsPp0hClYEtb2u1GDcgPN0nnYxuX9IBxmxgsXXLAQjsvuqm0tSk0uSmVBZ6ksJNjUc8_mqx_3ObyhFG2ICxGjP5s9fKapdkDgbrWzcDQ4nZ8MtPSe6LEAdhKL0wIaHQbEMSWQJZvdwI9u6_mFUV327-8q4u_PnjX-TNtPWxXiP_a6AZrmBOWqKVlKdMbWxGyCWteaDsY-XAwXEMcZStY3j8eloeO56n0sbgqe4xbPCjchL4dTP0JjpTrD58ten0XfVuBwjdH_Ny65rhvxdoN4R6rF2vIjQolK-Spk3tsAAA",
    "https://www.indeed.com/jobs?q=data+science&start=230&pp=gQFZAAABhdE7NrAAAAAB9eqlhwFZAQkBIkYzaA0iDRwJaAcQBy4HGgYAwbdyUNkFdLCLktRTxR-irJj8g-B_05l15k92lAvOWpDwqLVQUAS_0uW389Tvi3rybl9smCgY9momlsa_O47SSV-INcWMiYUiF2MyKDxSUMw7wplwMc9ToslmTMA43-OrLIhFYCIkJcozcRkdI_fzUjW7YzIwLZEoErXC5gA2dlsa3BGBB8_QJ3npWzXs9TtmeMfw11mAOHWmLonx5P9tMxSQf7uIJKPxLpE2tq_BoFJFBA9GyP-GMq-TW2dXgdPjQtpGiO6psKZXee6kze2M64tSl4JdEy5fYMM-yykL2vx6enEO1inKkRgzRjnnTwghTjROWsq_9OVTUio4Un0KGwHzr1qVeYxgI31OS_yNywlHGZhd3vZdNLQ9J5Lp8M9W5hAzIkbq9aHsUXrfxVkPs4QGLld1yx6brtm2G7J1I-0GV2-TAAA",
    "https://www.indeed.com/jobs?q=data+science&start=240&pp=gQFoAAABhdE7NrAAAAAB9eqlhwFhAQoBRo4BECINIg0cCTQGNgcQBy4HGgYDRHIEMDztXqQdLgVm5Ga9ESA-5IfjOBlMM2Jv6un-3kRJgfbkaiBk8bZ3TyKmT33RgPDQ-_Nk7gRaVwnHlyh8GVfZ7uawRHbiSnFUWi9WkFwISyhyiUGxwHUQkTHrjjLefy5Er3QsGhA3OhvxqTkLfDGM3MbvuN11nbLgkPQhvp2XUbh541bmPfBnQtZl4gWW9FrojLtKnirUaUvxxEcZhioxXokKltiURRbDewaa_Qm6ScVv2irFx7kb0i4-7R-fCqwOVdiXljQSuEA9UkBwtqQLf_hLHmyZjDBPtYHItNoSG9iud2yScna9wh3nkpCSd2OOgmVIRKnNs6PmjyK9D-XdsqMlx_MYtBOvvvOIrVz9bxD1_uepdtDHdWWhLB7eIIeGZ3nmOJkcMxY_ddFkXG2kR197gwNz92jOe8AOYZRFlzK58yvBDWYAAA",
    "https://www.indeed.com/jobs?q=data+science&start=250&pp=gQF3AAABhdE7NrAAAAAB9eqlhwFjAQoBRo4BECINIg0cCTQGNgcQBy4HGgYA1oKZcPrxGxdd6Nm-cdQzyWfYbVg8cHsT_OSvTUDv0r32n2OgtRHJ_MT0BeXQUy5MLqbjd37lGW6fh6ZmAdt2ezJbecJPdSP1pttR7Gj5wMzpzsiPZIsCMy2FQugYUFHTFhBr4Jjz0V61yo0qYwr463bSLqHWf1mx9cM-sKnM3QgGKEv_M4NU1OMDND_v5fRvRdHGT5ncauOIEDqp16XBMndq-jtBvOqAU45I_OzgX_Wc9sJtpZ_H5x0O7ygXHUyGHwWSK9W29ShY62tkzNKQjjikK09WNPwisA_lsBkOivhGxygNJwFTXcdH1dXGIFWjPvA5Fw0zTiir9JepAkarzRnXiucNtrJSPIoFDR6H-8qeCyHJpEtPt5C1SqmG9lCKh5kqneuz_nBOEeKZFkB3cUvTnFoCRtpAb3JEtQsBXdtnpC7P3fd4uqTxHAAA",
    "https://www.indeed.com/jobs?q=data+science&start=260&pp=gQGGAAABhdE7NrAAAAAB9eqlhwFpAQoBRo4BECINIg0cCTQGNgcQBy4HGgYh7wUgr2UVtQyxWKFKep5mfMSpapOc8LHiJE-kdbtWYPDca04OHIis7CQJMXPtwlgcgMGqjNJHVCpMppLTGAEsMJs4PIZjYpyqMuHfaI02UbrXJ3MCU36DPxX3Y6enMIlDB1GYLJKyZ-XwASzSP626I9vxvpadFAH_CxZCiisbIxEfa4L9_7j-5IcHYhLUzDKiMp4iVPpiattJQhfzqRAOEL9gSWWyCwAdIGBRA8R3dbPoAC5ZHRkfZtTBbZZQNxhN9zL8kt23TjroapT1PhKMzlpWNPEWgmvKNN4vjdeBnZUo2L91YceITvz7Ufsy_nsilQmItqQeCukjgeb74g59dI-o_alsN2NCo9sC7dnU3XdVOmE6qQiNmyG5sPv_QlhuJwMOUi3KSMHphvwyp_DKZbkB62KE9RYHEumCosWoEcnVj5v0HOBYwU-ZjyDz1JUB6gAA",
    "https://www.indeed.com/jobs?q=data+science&start=270&pp=gQGVAAABhdE7NrAAAAAB9eqlhwFsAQkBV7ABDSINHAk2BjYHEAcuBxoGFGxjJicM8wCsLE4yieNVNVHqmbrEcE13CSxX6iR6T8eIyMNHe0TJB-A7iqkVM8iykC7CyaT2NPDkEaBA0YSNmtWA3V7doVGnjSptwbX_Kd7lfDNOvuC4AW2wHnYVmnFcJqUdqQAEgtNKii3KGiyXOti6tDYuJaDdvM6cMsQwJj2MrMGYsy404lC2TLojlHexAovMH1hYZ-Q0qXHp-7GlNfIytzQn-DMil7PSldrLEsOQHrNW8MgfLgEAlfaTWc7eGi76LGPHRNQUx9drUq6DKmsihkDSkxRJ460clstwbcp5Boy2gBEpjIcYfzNlyAnJnW0tlfceB21PtHaINbdnu6grQNsmLYmnCLood_G_KS3TWZLj5ZHlDenCaHUUitQaiO8Fgoz3JK3D8rSN2JS55USxvVtuWczqA38O3Acl2ZyZBtsmaWnRgnUOMwQje_EfQ_CXGun05wAA",
    "https://www.indeed.com/jobs?q=data+science&start=280&pp=gQGkAAABhdE7NrAAAAAB9eqlhwFxAQkBV7ABDSINHAk2BjYHEAcuBxoGDJkUd1ldzVg21K1op6EZ-EH214IQY9Q530A95WNa6kTNYe3hwCsDG_eEnZ0cwieSXm2F8tziM_ogN0-s7O18mTcGx4juoho0lfaeNp7hEtU3iqzf5PdCyk1YJeJoARdU4vNFLBT0JACys46BIPW9AmZThXQU5RuUadgGFU-7YSN5taMos6mtwd6vzIupaBhVPD543mayqr9RyTGuKjKb4UjgzxZ3UNDm5KFpV506LARoMUT0kGwYYjf3DHU6KDEndGCP6ssE6oV34Wev68S2O-SrXqyKY1V3P01PqsD4iizqvpfSCvCWFxOhQs9gkjYSxNVgBCHmk2U8-_bsUIy0PEBWwUBCDXVyJG4IAkZen5NIBGgqQ7OgM3uz7ksFTv_11-epwegSwC6Zm_9zOPW9FhLGFS9iyFkTbauY4lOa_y5lM6Awv-75D3xE1X33Bul7K1LnunrTtVGRnCZ5AAA",
    "https://www.indeed.com/jobs?q=data+science&start=290&pp=gQGzAAABhdE7NrAAAAAB9eqlhwF2AQkBV7ABDSINHAk2BjYHEAcuBxoGB8Vb-AyOrHdjZ54njYqMO-hfastTICjVK4D4N4gI36b26O4_trpeMv2beedWYvWV4Wk60KpNKcgg-2y7gZoVKtkqsiqiRTaQ-t8au2v3wbyzhprYjUDDIYxuWraLYcMqvjVFqLkstLgc3_QU_UdTfnpXMeg3sIXFmYiF0u-zuhb9u5b02QYDQCdqTUDDUOQbEMHmtbeXgyJi7WGXJNe6ESnKsLXpgyALRzQdswlPKitn8NY_YAqLwkDj6NKaSexSONH1tgm2zrbjkBuIMpoyInYEpENtr3cUbLrYJwoIxNfB1GgDghRRAB_FgW0HcY6h2pVlQOLD7vzHSzMJXo0xmQUireaMAqYnqKJFT95a41CDiX1IDP9I8mJqr4ukGZlmtrfcYp7AidW7HxjHkGkXK07Ch2c7piN4GdXPAFoN9OZFUD19AxBVtxgyklWJ1OGFGGuZ-Fy-dwKIhq-8v0wCOs0AAA",
    "https://www.indeed.com/jobs?q=data+science&start=300&pp=gQHCAAABhdE7NrAAAAAB9eqlhwF8AQkBV7ABECINHAk2BjYHEAcwBxoGAOlBC0fXaX88s7wqXbtbh6nRujEQ_qOZoD1n4bEV8mwUbtGO28H_qZosxneS1ODn0xFKPjM9H4Hakrnrqz0pJgMJ3JF7QoW4Sf-C-TuG9WtU9BLSe4AmaR5kbP9mSqE8lrFF6Tb-YULrCvBJIIQFBZ56cMa6D01SSOLGz0JqM7_DQ0XQdxPvDZtBmBmBtpNUCmxs3Wsurir9efYSXXBpaTjY8edW_4Q8g5Bn0vxlFuq5ZXJ6_YIavIZecPcAFBT_Ctr3LKObaym18FopJwU3QVJCe_xiZjcSNEDIt_Wzk_lEJpAL4lG2vL02XpWT-cQs7asa8cfvQVjmMVst3K3SZzq-Qv7B7i4YLe4GiFsWgLfCc86gdAr1DAbaj1opDYrrDYrKSgNFmaRwUGT7UACmochMMPtCBl5zsIgSHI9wEBpc8uUd2gBB9jS195UKHSd4R0l-MVbqONEXEX1ck1toTRc4GuJYGfAAAA",
    "https://www.indeed.com/jobs?q=data+science&start=310&pp=gQHRAAABhdE7NrAAAAAB9eqlhwGFAQsBV7ABECINHAk2BhYGIAcQBzAHGgYsBxuKJSnbIiXtaQTBvIlnAId8UYCQQhTJ6kTGwJzSfeXxZqbsYRzAN9EOaJ1dGseLRlfXtkD5c0uMUFL2EPOWHHmjJVp8WcRIgWK5P7BsxGJFlq-p16OvDW2FOEQjSwJGe0oaM9PaaqoddiG5EdG4fVXPC2Yf8-QenaYdcXGsbyc9Vulq4CNmzK0gAE91eFtk0SIrvYAqBFXnPeN45Lskoo9awtKp69O7wZ-uCxCU5Tkzuc6Ug5w22XGraWY9auyjIp5Vn7-eDwoqRaHn_dXci_oSiELwNtN76JfoquA6cj7bKgfPNykq1v4BFk0x8nTJbrWRM8nXJpSGIM8jcn61A5SiVqpXfBx7Ty67h9Z8JXfn85ffg983nneqGbSOcgaqZ9OrNkDyiPExdzokJEnAiEtD66_71_sSy6MQ4hywbyJjTOMr3CPHvhpqbe-E8UDuG5o4pzJ4RT_OhnXW_D9xvswawL369-EhGv84GhMAAA",
    "https://www.indeed.com/jobs?q=data+science&start=320&pp=gQHgAAABhdE7NrAAAAAB9eqlhwGKAQkBV7ABHj4JNgYWBh4IEgdMBiwHNwLcsdZocg6tpToWG00eaGBEK0XAbvNr54T-EyOLsKc4JUIneucRbUV_4w6oXK_5L4TgqXIC4BIFOHO3oSKQ0VN9j2jmuKRk5yCHw1T7jNg2hLzvdtbg7atWKP6M3lk34ZUJy5D09LGNySV9wHgylyykyM1rAW4Sse03EPnkwWbMmDgncpT0OpyVI8aIsJIYuxbi32ECto1Hxds5cZamC7HpTujIBQyv52iVZGRae65Gk16zUjC6F5rAbn5w03zN9raQWAlVUjoiiUD_hesh1m-aLzR5B6nOgEXXp8yXsyY3rIgg7liuhw_DRzjfMiFseDbQvZCH8NkLxMF5RDxriUM0Q-XCldox-IT1lwt9YRRyjmABr7tfm7UaMfEDC3a6MV8Yl4uL9lSowOShcZ1BlvOWJKhkomYzE726Bwlg94b1dgoBxEUCaUDzJnqVI_3sA9Zoqe1eXYiduJ-UqRZmNvyBe35lIdxjBhWzdMcy_qgGhfsntgAA",
    "https://www.indeed.com/jobs?q=data+science&start=330&pp=gQHvAAABhdE7NrAAAAAB9eqlhwGMAQYBgKQCBhYGMgdMBi4HAb-RVT9yUkYynzfM2pww9T19rCYADWb8kJ61APHvd3ZSCP7jXH0S2a31nHW0VQVbxy-P0JoqD1fI3X-NFyh297Yz0t-0FpYifREb3-Q98-G8d0jf6I-8n-aSDsXHNvGy657neiM9g2iJdJ0WqQEWwUaJxPBsUAfuy3R-arjR2M_VeFlKQ50nplLdTVEqfFzu7NCdwx35mDSqrkte75bHeI-t35bf26wskFtEcMlS1u4B1oKtNpnetxuLCdiv_Ffa9nW6A0tIAxzFrSQ8yGXUI-IANc72j3C7WEAPAGOcw_CnzK-k_8i0zD13CfXBjkQDkcTUzuKUnUgz6iCFJ0_SJ4bkV4gC9gsNpwsREDKZDqSrqKYOUDrXlVRVil0pKZ8oFLqX8F154PISrfGCNryJumNBXK-Q6AnghEhujA3Krnljy49dVmr64tsloMnXhvdliE5QIA0ENrFd60aMRx_TJ7kzgX05bgUL4Byn8v3ppvFOy4k8u9PHnEZW3kXoAAA",
    "https://www.indeed.com/jobs?q=data+science&start=340&pp=gQH-AAABhdE7NrAAAAAB9eqlhwGRAQYBgKQCBhYGMgdMBi4INNfP26-6JlPCksTGZVrkuWLdMGZ6yTZ4AN6-Gr8i_B9elTvozBRjwsthtFMHybHXLU4R45ptpr8GIG7mbe8t0lQRyu7cUN6yTZd5fIDRP4YhGMq63_p-cCpuOiI6X8uLATskmDwwheLQXucW7qpNvzWjcPMW9lh9EGXPb0FMoc0Rpt3Y9oywHNyxvhfDb6j0xG7Mg_kDYfBBxIKThYQgZhmtP3FFON6NiGUsKzGjmsL4LJXCjk0JO8FQfZxvemR40lmHbveSZk1CwXIaB4wcQKpdB76eD9cTEPgAYC35r9YwTo-MVX89Rs5STbz8NFpWkmYQAPrR_EbIT5iZGhBgQkVBUd4sBUJUapzOLSte0a2adDY73BpjqpTb-l4DDYRA98mxdvaYr5W9Siktfgm9Z1OjTWCCAG6pSDnSJOGi9yBZYmkKfn835p1n-VHB1xEg35BKbTFXG2envHuk74UCgHRpLcLmp38CMgX4-WTuP4mD57XJxrhIn75e0qIjAI7UI8sAAA",
    "https://www.indeed.com/jobs?q=data+science&start=350&pp=gQINAAABhdE7NrAAAAAB9eqlhwGTAQcBgKQCBhYGMgc4BhQGLggBFBPqXGcQdKXuGOt8Mt8Hq5O0gx13V85wYJC_2MbIU3NkJhD1zwXLHftByGIiaEszsvwjK9snm33ybeBRkmsYoEsJ3zg0pPiBKE2oX6WcB8B_FBphS41IDC8SLIGT4_QCHt3e_qsELOJhG7kzfJBlOMSSCULmeLhKqg7OfOiCE05eTVDb38xqXWbteradyAFZHNPky_ulbFK9u49O1TtTSFDqPCDAA-5HRclxIvRdh764OgzzJAa_N87u_USOFyvAuFIrOAWyIATgXLl14SVWEemZTjngX5BgRMDSlW5aNrfsmjVogr2apxR3r7W0ixSHOLCRjFJu37nA3uKyAfvJFHn1aeTAhVtnnyLq3aNZlf4bveif3tZZTH6Npbkw_exX30N8Eow77bEkjiItFDEqoY_EwfdjObndzba8C6JUFxA8YK--ACXAsgAKVxnSvg3hXz-YZwVs0MsrgC2H7FzA245QGZsWO6V-5l957yIDY7VJUgk5EwCpPYzOq9JzhYpyEQAA",
    "https://www.indeed.com/jobs?q=data+science&start=360&pp=gQIcAAABhdE7NrAAAAAB9eqlhwGOAQgBgKQCBhYGMgcQBigGFAYuCAbRNpBbq2g0ElO9w08hwebQXNDxonsVyEOPyvUQQQ9IxIr_5ORV-rxLT3RdSumMWwMkTAy7svczHVTOfi6spYGZhQ3hXFJIUtMKSv-xq8BNqQ3aW2jBFuZk5qfKoWnli_FRA8Ze5BEq1MHGawmjDgSjWW7eXWYVJpK8n-vv50iz1FbUe6cwdupZC0d9VWSbH8QRuMsYj81q1xlg3QkC7ZpPskHdrfTBKVFkR678vKiqV_kdsG4Fp2oWein4uTZ_SB3p6lhWtaVfQmZyHCvrjq7xV5vHLq4I6ChYQAL0WidhJ06xDLIcp6crgsPOqAK2zIJz-_KIHUOpRmOJ5yMLumuwJPakxzYGLE6fO0E2oo04Dw6KswkQXDi1M2E5F5lGm07SiM7A9bTxdQgZ9rQL40kUmATIdhYxX2RtESU6UCzTkU54MzSMR67QMTEhYD3ceNYtSUhXJ2-ZCSHsf3I7QUP_T5XzGwxqjlGhezZV_a2V4io91u0fTusW108AAA",
    "https://www.indeed.com/jobs?q=data+science&start=370&pp=gQIrAAABhdE7NrAAAAAB9eqlhwGQAQkBTZwBMogBBg4JOAcQBigGFAYuCADedB13mZMEjRM19trog4YsPOEJRRklMj1pd88aTsszdNCq4NbA72uVvqHl8pYLv_F2m7g9rqYYDnpX1SMYfOXJ9mplG9smgzocIKnBYF2qqqHbUaU97JbqY4_ys8rkFkWCdPEgbTiZzB_srLtRZKBojrl9d5YFbO8RJIC7Dk3TwCtocmuGom6eRmLpHFy4nbOhnRrEt_oADAu1uywUmgFbujlDSmA8fJ3LpFNqRDG8nomKgzR5aWO_QH2Ho3Hx8Zxi2YS9dxyVGMgmz_YiN-v68aVFG01ql-6Uic5esT8LLfYp_Jbe5PM_4RkWd2bE4HLdyQ7ray_4Z5_iyXJHGdzqJ6JUCvijwN03YlyqzF3HY0oTQ3TOG0mVLsq7STFWV252qFWSlwdCCb_4FR6ZY_ofXam-Mq2Er5D9b1XPvZkG5dh5hV_BKnX1zgxYimlZkxuZ784Mwp68TukwK5WvNQIDe9_nLtP7fCSTMpbce6yvZu8FmtCdPQO5CwAA",
    "https://www.indeed.com/jobs?q=data+science&start=380&pp=gQI6AAABhdE7NrAAAAAB9eqlhwGHAQcBTZwBMogBEEYHOAYUBi4IRQcQwOgxOd8PfcvvWaBf1Kq_tg8ia7AQ4PZV0JV4CERAwAi2xVBpif6eiKZwql5OMtdrowR_O4Nt8R96RNbsEcvYIvL-7lZEcw4oWkVSVe_tOHSanvEdLOe_Fs0l5wggJdIVBut1pDpC7wwUJmOCApjRqsQpr_SS7PveCCC56S0nP-tCJlGLBgBmarvOfRp8YnSXaUUVcOICtyTP0wxJcQOD8VrJQoQy_ElDvog2h6Nsl-OtYls3FoEPEaPttKKAypR0uPg-YdhojLv1M9EgDJUOZWGXHR9ajpORlub6q31mX7AdcIxSPE9ynMEFUPNi3ZgHNMGSB_utGugtJfRbE5Gt3_O8wyoDbLy4WzhC0Izsvgu2c_2d3IIfyf22Djf-xli4iKU-JvwKPk7IiaDHbUkbFP5hNyGd5bJf1FCw5x0QGlZhEbIjZgPTj44uwubdRAuuNy8_ccF7ylyXJa5FDfHraKsJFxQAPSVB_JR3mFwMk6bydgAA",
    "https://www.indeed.com/jobs?q=data+science&start=390&pp=gQJJAAABhdE7NrAAAAAB9eqlhwGFAQcBTZwBMnAHGBBGBzgGFAYHXCC-sVhkLB4Jtu-XeSVnnfsuf3e0WYWu5wzvGMS-ld3JPcHw2XigKl_4Po1AvDd6sOAjdD_cyNaPQ5sYBtDAZswfSsL3zs3h18djGSNxOLSeWC26YCezIXc367wGoKLuzuHXKYZ7ZVKyttNLznRW9755KePuC6MhilAkIZTkFW7Q7i9Dw-WfWbvpFKIrX98DGZI4FnHXnriSCdGtlg6C43enQjmAidpJH5lzxk6AcykIvdp7vNooikfsL5LPg6VDnd4kFioxZFqWu3HulYl7_E7hV3wRpXZDHyvVH3I77Wp1g84hfxxBBd5SLi9ARjUoGNOIwzFLwLcSncjW0pG_q1EwY360osRU4A-SRuzeO1_O8JDM18FgcYhI_OhUNGasGWk69YpVciSPbu_xVb186MczxNSyIGZkuLKV9Tz3W-DxW8T4wLN4PGNqSJwseY6yobZZ5VLE-37iXZ1PXNB5sbVjCRhs7i0bpkrIH1yIgsMJpfEAAA",
    "https://www.indeed.com/jobs?q=data+science&start=400&pp=gQJYAAABhdE7NrAAAAAB9eqlhwGIAQgBTZwBL3AHGBBGBzgGFAY2Bkh_hQAqcl0UolLXdT9bJDvfgDzBr_htp1V4Cg_QtnCwaJDnCblpAS34KfXPbvdsX6phu8wslf-7Txrp18cedYyMB84RfM3bJorzpAmfJZZIuInb-y04TLsPDkHy23rWTAJ7VE_RhUPTkm35zF7ZPRHTlsHsxc3QFd7tT3hPwsBEwTeAUunC5NybQJmU5BbWRo11dji6WjO-pKMFIQTvxPndT0aCVCCT0i2GYrQEm8-IKZIVV7bQGjx7ZCaN3PkqXDLpA0fWQROp9HI2qzNh7txEvg1naeVbPxakiBXxNM2QmfbsKfcfddrgbhofgHXyc-FfS-SonFsszqe9BIw_tK2yhnrgXxJI7RoeHwKDQGtbHls5PNNZ0yqAaZ4f_R3U3eO_nn-tXmQr505Gfi06By01vigza67gvUELAGaB7JSQkyfAbPbnpwdqgI_MFDdT6OawM3y2-SB5O8VLZZqb8kK4LuJpdbXxJZq7361YKxlDcHnByW4AAA",
    "https://www.indeed.com/jobs?q=data+science&start=410&pp=gQJnAAABhdE7NrAAAAAB9eqlhwGKAQkBTZwBL3AHGBBGBzgGFAYOBigHSH-FAFINYlK6SAXyjeoxn2QFkhQrmXXhThLlF7eSp0XtJdFJTern_psE1yFNFRoVLp4W-dCURfagRE4mbw3b5pPfk5iQ6uyMvYExG358rM5PO6VYEe381SwRFOkdpJnc9A127CuAHab5IFG8nwrSYAQ2IwFJUdni-_fqkukXmYqcY8AxD06RbmwII7tFSKABmUdeyHri0yX0FxBZJMM7mK3RmB4BvDNxELIGuMPTqmDxuLoJ_kQ7o0avyk7FC9GSpgSJ6UXBEex1L4xCmokKDsfZH6ViE1CSyC4lH8_C3SLOxIjPXsEw58bSPca_WJRsH5qJ1yyuyc7mM_h7itjcjNJEHtQYRjpxwLjX0B91LxFnOqcKuooYICGp21enmpg8U5_NoU811rABCN2IIA0Up0eWGIhaP6uKabsJhgKeZ0XJB9U0L7JTHlPx4wdSvPg8pVP8V92rbdWA_qeSPFoXIMWitbgMacnWwxsQDZgVSsuwNR_dowAA",
    "https://www.indeed.com/jobs?q=data+science&start=420&pp=gQJ2AAABhdE7NrAAAAAB9eqlhwGQAQkBTZwBBg4oehBEBzgGFAYOBigHATMn_MAI4JR6w2y_3pElWWwTI6YQOxD5r3X7wE_9pI07JQu_-WKq0Z7Ou0CymQ3NSuapCKajrhoCERtSyVGK4RX9g4QvstCjNjIDZ4VBsF2GfhrGM93vrc5d6L24fG7cXseYUKrNrr04XoBnTabUqj9zM4zQsSxU55lgsFBeDR9tzOFncPL_kyDIqdjZL4fuXP9bEUln9MhJzcu8RU5gGyH4C6ia7H2IGvylsRHy9Vj3J9HcqbZcopbBgbfxVbVe4vB_N67RH2QJNncR3vDA0xZZFjLUlrg7bd-NeVpVVCLDFdhKci6OAtfzC0FX-aZYBSuq8S4KsInkUgwgO3XL8D28RcpZbrbEVWKBhMCMVdBk3cvWJX_tRonXte4rkT4P4oTXoD2lnvV-lxFR62j0Q9ZmBytGEccpJXbPbD7PJYIpKM71xgzwFvMM9JDl-esa4DH3Kgt_JSbjKD_SK93K0l2v35mglWeFgnTv-jVRpw2pUxhnXSZhEeDb4wAA",
    "https://www.indeed.com/jobs?q=data+science&start=430&pp=gQKFAAABhdE7NrAAAAAB9eqlhwGPAQkBTZwBBg4oehBEBzgGFAYOBigHK-EkG277guz3WK2yFLwxWJUFF7ktAmzP7EiohVzltI3HEzgs8aN8utsCd5Ez3A6q4LWwyB5jby0tJ-wKvINWVxi24KwhhD5HoYSAmldsQpGwM9N654mpCiL4POFbLYer75K2eM0LvidFeFLKF3cIDeyOditAJEINUWHrJYWq-dIr-Yu6NHIW-0Xk5dvJo2_frjt0CP9bs_QJI6N-KvSc0_YLrotJlrNcfFMhgZsOsKogZ5VtHzomEVz7JztXcZdlB9WQeZpmx7ph40A9JD3f-vHTDEiKqyQm5wygFKOV7M5j9o9MixCBn3G82ER_tmkPmnQh8Kwg-zf5VEdS0bf9vX2kKkr1NGs8gdJL6h_s0hSfkj-VF7pOgPQS0rTjpPg2Us5zVtIOIpP6tI_8XLKWLXeAq8ERvrgwt9JwV6qiicG0vg2bx2D3_U3uUbtHrwQkYT3jPHPdDF2K_8Boycsw6l_nunKIaXjK8qRaFNx0vFiWKKD0fzAj6ptPAAA",
    "https://www.indeed.com/jobs?q=data+science&start=440&pp=gQKUAAABhdE7NrAAAAAB9eqlhwGNAQoBTZwBBg4oehBEByALGAYUBg4GKAdfT5iKUphDR__frMLeyTN3VRmYmgACMW-W4VXtcIyV_bm4nffrAl9HmIxQXZtDYpJ7t4RQ13229h_9Yo_UZRdFjCcu1pTFPP2kMCHfgrxj0sb8iWNyBjN8uQYONIECFmMmrDsBI69yIyEAV_TmCocDiDnNz2equTjAIjXn-3JGy0mVRK3m3Lgy4m588uJ3LL70Xw5uOd2TnnBlZFL7Hb1IH2geBkvikNcu-hOapk5ZBB1z4sg3tf9g82nQyq48_1rpXmyhs1QgIcO03kcIO-5sYGIzKO2UauJFP9Diu91AGwPWqkPQnWhqHa34gfQj4EbazOBDzHtcRES5p-ToR5TrryNQItz-u8fydCDJLWobWjE6Y9wvK4SCKR6uokSH-fGHzE_XBLyrNGO2sln2T1tTvF7G1CV-zeF_OCw92Bdf9wrBnIqi1H0DUToxGgI279bNPYKH7vhYbQfYYlxwkkxU5xJDl3ozco-ytHeGhFLvLy3UxpEMYAAA",
    "https://www.indeed.com/jobs?q=data+science&start=450&pp=gQKjAAABhdE7NrAAAAAB9eqlhwGOAQoBTZwBBg4oehBEBygHEAYUBg4GKAcCmy0ryEIp1w4PBm2kLEpAyyTWny1kqYZtM2ip1Kq0fNUx-9x7lo9ggs8ZaEOyD7097kbd719B9o3DK33UMjRv7TtnA6OkS0WGspSwCzWHqQKCCOxy-0w69A7kICoUOEKkaqGKqgnm3XnmLR-9pIngmOMzI4BO3NkcmjvEQPwLZN1iPaPmcGoOSv_g8JLqI3vNGs4eqUBwuuCGwL_ifoLTH1EiT1I8Y_v1DcGYYjwuAgtNhQ9qsx3vGkiHmRi12FjQlzuVY8SV9msNwH4wqjUUxPi8GyySjwnaZpb01rJKIkfWMJ3ZobWsERcmVFLuGiiO-aRljzhu0nBypxeCTp3GS3oadDCzBHe_xgzq7qDJU72YLm_MBRK4VbNspaAciU6X45YhPElpHbARwvd5rd3ToHGBXwXDoD8ZxOZppe_WI7ghaYXL-JAZopaE0qGRnkwmQfxeyXZl3L1f386GBVggS84aBoimTI0CR1dPTd1t933PPZt4bG4AAA",
    "https://www.indeed.com/jobs?q=data+science&start=460&pp=gQKyAAABhdE7NrAAAAAB9eqlhwGOAQoBTZwBBg4oeg1ECCgHEAYUBg4GKAcSPrvVQHf5ycURpZTVO-VsoTxHzsrLQmMOqnXsDTesK1yjdCUo-78gN8IVDzc_Wg1twMoY_bLCp43LlWsmXoauPHJIMtQeC7pXJwJJ0KZHywHe00ouk0Gep-EgCDQL5WHo-W5aiEiWVjUIO_lXMKMVChVXsUdhoy-9TwzuNyazgeFYg_MkTa6IoISLCQRq53I26QdUR-cLhpTmuf3-gFjJtDF3WO8vbExtw3SvgsROH63MlL82UlOXPYsyqRC7wdU86dWdpXWAHUS-IN48RdODYYhsK1oG2Fbk7g51LLUniO-uvN7wYWp_ybwTeD7ycZ1GyrqH84-AV2Syp4295_LK7j1WroC2bNhwmwKOiThq4FbJ1BPi1SxXkZW-SxKplgMUkq4d2vI6HC_pzd9lWd41g84rrLdFiWHE608jyYu8C5zMi2rrjFibA_S6b2tMivsNIZKkOUkGhMFuUEEHdSE_O3f8fJNKdQr68VsWLBhL6Io11nzOMsYAAA",
    "https://www.indeed.com/jobs?q=data+science&start=470&pp=gQLBAAABhdE7NrAAAAAB9eqlhwGLAQkBTZwBBg4oggEJPAgoBxAGFA02BwFzsC0TgCwHlbhdU_gmGfZMvy-AuNu9FGT9cF_vmcTziCmZ0lACZIPgLs32U6xWLdbserGcvz0ng-z2apoQMqyOjjaZZrjNLuIu2giNah5uFMtflzWYrkZnlp6qooXhYuEzpi3Tbl1wHNwqh-Y74i45Tw6q5tUIKr9vVqH4MyOTVhZg7H-Q9uTml1eA78LVqwqD7yNJVGkBDNE7TFNFCB3UX9pDldqLH4AyAgBKIcu5T51oP4tPz3iJUFyt9Z-dRLablAoebVGFS3_nJwkKj_2-NvQnyuladXJ-eT-9coE1gvhluYcH-_xJDT6g-jXtbRXefXoeIlqN8Y80RDgdQg8KyGtZ0oyPqpYf7uyMQKxZNGTKrGc67bjbiCu7B_MUrQuHmLK8ic8L0NBmO1dvlVc1MXzDadN77iO4ArW3NLkJZs50V5KgKEWzMy6ZzCjCgGlRsbPXnTnSrtwcXsxfAo4DVMew3R8HjX8PeO5AY2q7BVNLlRUAAA",
    "https://www.indeed.com/jobs?q=data+science&start=480&pp=gQLQAAABhdE7NrAAAAAB9eqlhwGMAQkBTZwBBg4oggEJPAgoBxAGFA02B19SM19bQ8ql9nr3M8iNcnkom_LylstLb3pXC6aiPYC9RX0LOVy5Ii7AiaMe8F7Sh4EFqzGvsMrjWIIKhQUsSrSNMgDAFkiMdJKvYfyU3bnxkIqT85XihJUGiuLHgyxdCjp4z7jeY_6bt8bMWu_skltTrsebJe5kT2I4Dy_CPUZD1CYiULK3gKJG4sXRbniHcwOlTVC-WffC9PquxvASMXd1Ogvoijfb9gpeXmC8mQijPKWa_4yG6xq8gznUkxXoSP1-V7BcJzmzon_sKyqX1aWoEwLDaXGXFVbIZf_xUaiebkt5w1JH38ZtaPwDKFkHo1mvuvjlwqebMNewRVGkZ6_NQNdMUEjXVZRZuHkgOk9LtPIr48yMpQ53kmJmaCidE00Nw2XBoCPm76zukxEcTcOoeV7_OTnyp1x65zyCAxyKLC0PNhal5WzwQQVjLISPLzNTO4PwntKpd-Zn2ve6r7kquu3OgCiRx00ItCtyM0rWgx1rVu6TAAA",
    "https://www.indeed.com/jobs?q=data+science&start=490&pp=gQLfAAABhdE7NrAAAAAB9eqlhwGKAQoBTaoBKIIBCRQGJgYoBxAGFA0kCBIHAJfS2rUsrk1rY3C6h6CmMyZZ5nOhlRCLgWE17yYBQH_GruKSElawi7FsFZo5SvTQkXaCwiHpO9kBb8mkWjh_8JcC8ompN4TXZeBBzZ2qTdUhtl41XE5PZEB35fnD3dQnVENRrfVIApLzKyZMFLD1yAqfm5FcXgmXl6H-MqMQkQ6bUI91zu_Zak3VWhmH_VD9fjLXVofgbE8slVURQ2UqvnGJi8RIogxZ2pIQ3Kmpg8eXDsWj_kb0cluPdSP1g6-SZj8LLQ61uVa1B3MR6ZaJO1RvmU8HW1JfYEGVoz6m6KaUBMMPyMwKssErtNC6WvTNiU61jiYjmVJhY4Q262cWoY1NB3NSe6tdVIrXYIEPpqAVmYgPO_M1fly4g9Xd1iv6xvvXYiVshG_qiADKFcpCG0EBhzpEmNzvvqOnk9pc4oG6BIusnMecHQ_KWW6fLUdGqCW-cP5C55_N_kSLzWqBit0F_Jg0VHolu-SyvtHNdqk9XQAA"
]

<ul><li>Now, let's finally start rendering the webpages and picking out the tags we want.</li></ul>

In [3]:
postings = []
errors = []

for url in url_list:
    # render the page at the current URL
    driver = webdriver.Chrome()
    driver.get(url)

    # first, search for the list of all postings on the page
    postings_list = driver.find_elements(By.XPATH, "/html/body/main/div/div[1]/div/div/div[5]/div[1]/div[5]/div/ul/li")
    
    # there are some empty <li> tags that need to be disregarded (same positions on each page)
    postings_list = postings_list[0:5] + postings_list[6:11] + postings_list[12:17]
    
    for posting in postings_list:
        try:
            try:
                # grab employer name
                employer = posting.find_element(By.XPATH, 
                './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/span[1]/a').get_attribute(
                'textContent')
            except:
                # employer name is not a link in this case
                employer = posting.find_element(By.XPATH, 
                './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/span').get_attribute(
                'textContent')
            
            # grab in-person/remote status
            in_person = posting.find_element(By.XPATH, 
                        './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/div').get_attribute(
                        'textContent').find("Remote") == -1
            
            # grab job title
            job_title = posting.find_element(By.XPATH, 
                './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[1]/h2/a/span').get_attribute(
                'textContent')
            
            # grab employer's address
            address = posting.find_element(By.XPATH, 
                    './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/div').get_attribute(
                    'textContent').split(" in ")[-1]
            
            try: 
                # grab estimated salary
                salary = posting.find_element(By.XPATH, 
                './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[3]/div[1]/span/span').get_attribute(
                'textContent').lstrip("Estimated ")
            except: 
                # salary is not estimated in this case
                try:
                    salary = posting.find_element(By.XPATH, 
                    './div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[3]/div[1]/div').get_attribute(
                    'textContent')
                    
                    if salary[0] != "$":
                        # WebElement displays full-time/part-time status in this case, instead of salary
                        salary = None # give up searching for salary
                except:
                    salary = None # give up searching for salary

            new_posting = {'employer' : employer,
                            'in_person' : in_person,
                            'job_title' : job_title,
                            'address' : address,
                            'salary' : salary}
            print(new_posting)
            postings.append(new_posting)
        except:
            print("Couldn't scrape", url)
            errors.append(url)
    driver.close()

{'employer': 'Booz Allen Hamilton', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Camp H M Smith, HI+14 locations', 'salary': '$58,300 - $133,000 a year'}
{'employer': 'Bloomberg', 'in_person': True, 'job_title': 'Applied Statistician - Data Technologies', 'address': 'New York, NY 10261', 'salary': '$160,000 - $250,000 a year'}
{'employer': 'Megagon Labs Inc', 'in_person': True, 'job_title': 'Hands-on Technical Product Manager: LLMs / Data Science / AI+NLP / Research', 'address': 'Mountain View, CA 94041\xa0(Old Mountain View area)', 'salary': '$185,000 a year'}
{'employer': 'Arcfield', 'in_person': True, 'job_title': 'Data Scientist Senior Technical Specialist', 'address': 'Reston, VA', 'salary': '$118,515 - $284,791 a year'}
{'employer': 'Enbridge', 'in_person': True, 'job_title': 'Data Specialist', 'address': 'Houston, TX', 'salary': '$96.5K - $122K a year'}
{'employer': 'Celink', 'in_person': False, 'job_title': 'Data Scientist', 'address': 'Remote', 'salary': '$80,0

{'employer': 'EY', 'in_person': True, 'job_title': 'Government and Public Sector - Data Analytics Solution Lead - Senior Manager', 'address': 'McLean, VA 22102', 'salary': '$142K - $179K a year'}
{'employer': 'ROR Partners Inc', 'in_person': False, 'job_title': 'Principal Data Engineer', 'address': 'Remote', 'salary': '$150,000 - $180,000 a year'}
{'employer': 'Incorporan Inc', 'in_person': False, 'job_title': 'Azure Database Architect', 'address': 'Los Angeles, CA', 'salary': '$75 - $80 an hour'}
{'employer': 'Ryzen Solutions', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Foster City, CA', 'salary': '$80 - $92 an hour'}
{'employer': 'Tesla', 'in_person': True, 'job_title': '\u200bData Analyst\u200b, Reliability Engineering', 'address': 'Fremont, CA', 'salary': '$111K - $140K a year'}
{'employer': 'Mayo Clinic', 'in_person': False, 'job_title': 'Data Science Analyst - Platform - Remote', 'address': 'Rochester, MN 55901', 'salary': '$95,493 - $133,682 a year'}
{'employer

{'employer': 'FlexIT Inc', 'in_person': True, 'job_title': 'Data Science Engineer', 'address': 'Beaverton, OR 97005\xa0(Central Beaverton area)+1 location', 'salary': None}
{'employer': 'ERPMark Inc', 'in_person': True, 'job_title': 'Chief Data Scientist', 'address': 'Richmond, VA 23229', 'salary': '$65 - $70 an hour'}
{'employer': 'Mitra Chem', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Mountain View, CA', 'salary': '$136,000 - $155,000 a year'}
{'employer': 'Included Health', 'in_person': False, 'job_title': 'Senior Data Science Manager', 'address': 'Remote', 'salary': None}
{'employer': 'Booz Allen Hamilton', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Camp H M Smith, HI', 'salary': '$58,300 - $133,000 a year'}
{'employer': 'Bloomberg', 'in_person': True, 'job_title': 'Applied Statistician - Data Technologies', 'address': 'New York, NY 10261', 'salary': '$160,000 - $250,000 a year'}
{'employer': 'Megagon Labs Inc', 'in_person': True, 'job_title': '

{'employer': 'Topaz Labs', 'in_person': True, 'job_title': 'Lead Software Engineer, Video AI', 'address': 'Dallas, TX', 'salary': None}
{'employer': 'Novartis', 'in_person': True, 'job_title': 'Associate Director, Data Science (Biomarker, Omics)', 'address': 'Cambridge, MA 02139\xa0(The Port area)', 'salary': '$192,000 - $288,000 a year'}
{'employer': 'Battelle', 'in_person': True, 'job_title': 'Network Security Researcher (Senior)', 'address': 'Chantilly, VA 20151', 'salary': None}
{'employer': 'Dataman Health Inc', 'in_person': True, 'job_title': 'Senior Data Scientist', 'address': 'Melbourne, FL 32904', 'salary': '$54 an hour'}
{'employer': 'Intermountain Healthcare', 'in_person': True, 'job_title': 'Polysomnograph Tech PRN', 'address': 'Wheat Ridge, CO 80033', 'salary': '$22.45 - $38.59 an hour'}
{'employer': 'Anthropic', 'in_person': True, 'job_title': 'Data Analyst', 'address': 'San Francisco, CA 94111', 'salary': '$200,000 - $280,000 a year'}
{'employer': 'Talentech Consulting L

{'employer': 'Peraton', 'in_person': False, 'job_title': 'Data Engineer/Architect', 'address': 'Monterey, CA 93940', 'salary': '$112,000 - $179,000 a year'}
{'employer': 'Archon Resources', 'in_person': True, 'job_title': 'Senior Manager Data and Analytics', 'address': 'Catoosa, OK', 'salary': '$165,000 a year'}
{'employer': 'Regeneron', 'in_person': True, 'job_title': 'R22391 Clinical Flow Cytometry Data Analyst TEMP', 'address': 'Tarrytown, NY 10591', 'salary': None}
{'employer': 'Ryzen Solutions', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Foster City, CA', 'salary': '$80 - $92 an hour'}
{'employer': 'Tesla', 'in_person': True, 'job_title': '\u200bData Analyst\u200b, Reliability Engineering', 'address': 'Fremont, CA', 'salary': '$111K - $140K a year'}
{'employer': 'Mayo Clinic', 'in_person': False, 'job_title': 'Data Science Analyst - Platform - Remote', 'address': 'Rochester, MN 55901', 'salary': '$95,493 - $133,682 a year'}
{'employer': 'Nagarro', 'in_person': Fa

{'employer': 'Pueo Business Solutions', 'in_person': True, 'job_title': 'Data Science', 'address': 'Quantico, VA 22134', 'salary': '$145,000 - $165,000 a year'}
{'employer': 'Mitra Chem', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Mountain View, CA', 'salary': '$136,000 - $155,000 a year'}
{'employer': 'FocusKPI Inc.', 'in_person': False, 'job_title': 'Data Analyst', 'address': 'San Francisco, CA', 'salary': '$30 - $40 an hour'}
{'employer': 'Booz Allen Hamilton', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Camp H M Smith, HI', 'salary': '$58,300 - $133,000 a year'}
{'employer': 'National Security Agency', 'in_person': True, 'job_title': 'Data Scientist', 'address': 'Fort Meade, MD', 'salary': '$77,745 - $183,500 a year'}
{'employer': 'The Travelers Companies, Inc.', 'in_person': False, 'job_title': 'Data Engineer I (Ab Initio)', 'address': 'Hartford, CT 06101', 'salary': '$102,600 - $169,200 a year'}
{'employer': 'Softdev Integrations', 'in_person': 

{'employer': 'Enbridge', 'in_person': True, 'job_title': 'Data Specialist', 'address': 'Houston, TX', 'salary': '$96.5K - $122K a year'}
{'employer': 'Mayo Clinic', 'in_person': False, 'job_title': 'Data Science Analyst - Platform - Remote', 'address': 'Rochester, MN 55901', 'salary': '$95,493 - $133,682 a year'}
{'employer': 'Topaz Labs', 'in_person': True, 'job_title': 'Lead Software Engineer, Video AI', 'address': 'Dallas, TX', 'salary': '$117K - $148K a year'}
{'employer': 'Garner Health', 'in_person': False, 'job_title': 'Data Science Engineer', 'address': 'New York, NY', 'salary': '$110,000 - $145,000 a year'}
{'employer': 'Adtalem Global Education', 'in_person': False, 'job_title': 'Director, Data Science', 'address': 'Remote', 'salary': '$97,254 - $175,762 a year'}
{'employer': 'Mitra Chem', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Mountain View, CA', 'salary': '$136,000 - $155,000 a year'}
{'employer': 'Booz Allen Hamilton', 'in_person': True, 'job_title': 

{'employer': 'PAYLOCITY CORPORATION', 'in_person': False, 'job_title': 'Data Science Manager II', 'address': 'Remote', 'salary': '$153,250 - $194,259 a year'}
{'employer': 'Workday', 'in_person': True, 'job_title': 'People Analytics Data Scientist', 'address': 'Pleasanton, CA', 'salary': '$111,200 - $192,200 a year'}
{'employer': 'Booz Allen Hamilton', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Camp H M Smith, HI', 'salary': '$58,300 - $133,000 a year'}
{'employer': 'Capgemini', 'in_person': True, 'job_title': 'Senior Applications Consultant - Power BI Developer / Visual Analytics Consultant', 'address': 'Atlanta, GA 30328', 'salary': '$115K - $146K a year'}
{'employer': 'The Travelers Companies, Inc.', 'in_person': False, 'job_title': 'Cloud Data Engineer II - Enterprise Analytics Data Products - Enterprise Market Research', 'address': 'Hartford, CT 06101', 'salary': '$113,900 - $188,000 a year'}
{'employer': 'Computer Enterprises, Inc. (CEI)', 'in_person': True, 'jo

{'employer': 'TEKsystems', 'in_person': True, 'job_title': 'Machine Learning Engineer', 'address': 'Salt Lake City, UT 84101', 'salary': None}
{'employer': 'Anthropic', 'in_person': True, 'job_title': 'Data Analyst', 'address': 'San Francisco, CA 94111', 'salary': '$200,000 - $280,000 a year'}
{'employer': "O'Reilly Auto Parts", 'in_person': True, 'job_title': 'Supply Chain Analyst', 'address': 'Springfield, MO 65802\xa0(Bingham area)+1 location', 'salary': '$48.4K - $61.3K a year'}
{'employer': 'TikTok', 'in_person': True, 'job_title': 'Data Analyst - Tiktok Ads', 'address': 'Seattle, WA+9 locations', 'salary': '$114,000 - $209,000 a year'}
{'employer': 'Eastridge', 'in_person': True, 'job_title': 'Customer Service Product Complaint Specialist', 'address': 'Aliso Viejo, CA 92656', 'salary': '$45 an hour'}
{'employer': 'FlexIT Inc', 'in_person': True, 'job_title': 'Data Science Engineer', 'address': 'Beaverton, OR 97005\xa0(Central Beaverton area)+1 location', 'salary': '$114K - $144K 

{'employer': 'Booz Allen Hamilton', 'in_person': True, 'job_title': 'Data Engineer', 'address': 'Camp H M Smith, HI+14 locations', 'salary': '$58,300 - $133,000 a year'}
{'employer': 'PAYLOCITY CORPORATION', 'in_person': False, 'job_title': 'Data Science Manager II', 'address': 'Remote', 'salary': '$153,250 - $194,259 a year'}
{'employer': 'Humbi Analytics LLC', 'in_person': False, 'job_title': 'Healthcare Data Scientist', 'address': 'Remote', 'salary': '$80,000 - $120,000 a year'}
{'employer': 'AMP Robotics', 'in_person': False, 'job_title': 'A.I. Data Quality Control Technician', 'address': 'Louisville, CO', 'salary': '$15 - $25 an hour'}


In [4]:
import pandas as pd
df = pd.json_normalize(postings) # flatten a list of dictionaries (JSON format) into a DataFrame
df

,employer,in_person,job_title,address,salary
0,Booz Allen Hamilton,True,Data Engineer,"Camp H M Smith, HI+14 locations","$58,300 - $133,000 a year"
1,Bloomberg,True,Applied Statistician - Data Technologies,"New York, NY 10261","$160,000 - $250,000 a year"
2,Megagon Labs Inc,True,Hands-on Technical Product Manager: LLMs / Dat...,"Mountain View, CA 94041 (Old Mountain View area)","$185,000 a year"
3,Arcfield,True,Data Scientist Senior Technical Specialist,"Reston, VA","$118,515 - $284,791 a year"
4,Enbridge,True,Data Specialist,"Houston, TX",$96.5K - $122K a year
...,...,...,...,...,...
460,Mitra Chem,True,Data Engineer,"Mountain View, CA","$136,000 - $155,000 a year"
461,Booz Allen Hamilton,True,Data Engineer,"Camp H M Smith, HI+14 locations","$58,300 - $133,000 a year"
462,PAYLOCITY CORPORATION,False,Data Science Manager II,Remote,"$153,250 - $194,259 a year"
463,Humbi Analytics LLC,False,Healthcare Data Scientist,Remote,"$80,000 - $120,000 a year"


<h1>Analyze the Data</h1>
<ul><li>Ok great. Let's try looking for an in-person data science job in California that has a salary upper-end of at least $90,000, for example.</li></ul>

In [5]:
# rename the index
df.index.names = ["index"] 

# query for in-person jobs
is_in_person = df[df.in_person] 
print(is_in_person.shape)

# query for jobs in California
is_in_CA = is_in_person[is_in_person.address.apply( 
    lambda addr : addr.find("CA") != -1 # address contains the string "CA"
)]
print(is_in_CA.shape)

# consider only postings with listed salaries
salary_listed = is_in_CA[is_in_CA.salary.apply(
    lambda sal : type(sal) == str # i.e., not None
)]

# convert salary strings to integers
def formatToInt(string):
    formatted = string.split(" - ")[-1].split(" ")[0].lstrip("$").replace(",","")
    if formatted.find("K") == -1: 
        # there is no thousands (K) prefix in this case
        return float(formatted)
    else:
        # there is a thousands (K) prefix in this case, so we remove it and multiply by 1000
        return float(formatted.replace("K","")) * 1000 

# query for salaries over $90,000
makes_at_least_90k = salary_listed[salary_listed.salary.apply(
    lambda sal : formatToInt(sal) >= 90000
)]

postings_of_interest = makes_at_least_90k
postings_of_interest

(313, 5)
(106, 5)


,employer,in_person,job_title,address,salary
index,,,,,
2,Megagon Labs Inc,True,Hands-on Technical Product Manager: LLMs / Dat...,"Mountain View, CA 94041 (Old Mountain View area)","$185,000 a year"
12,Tesla,True,"​Data Analyst​, Reliability Engineering","Fremont, CA+1 location",$111K - $140K a year
14,REVOLVE,True,Data Science Analyst,"Cerritos, CA 90703","$90,000 - $120,000 a year"
16,Megagon Labs Inc,True,Hands-on Technical Product Manager: LLMs / Dat...,"Mountain View, CA 94041 (Old Mountain View area)","$185,000 a year"
20,National Geospatial-Intelligence Agency,True,"Imagery Analyst (Offutt AFB, NE)","Vandenberg AFB, CA 93437","$69,712 - $129,164 a year"
...,...,...,...,...,...
429,Mitra Chem,True,Data Engineer,"Mountain View, CA","$136,000 - $155,000 a year"
434,"Carmel Partners Management, LLC",True,"Analyst, Data Science","San Francisco, CA 94111 (Financial District area)","$105,000 - $115,000 a year"
441,Anthropic,True,Data Analyst,"San Francisco, CA 94111","$200,000 - $280,000 a year"


<ul><li>We managed to find 89 matching jobs. Nice. Let's check for null values:</li></ul>

In [6]:
postings_of_interest.isnull().sum().sum()

0

<ul><li>Perfect! It's pretty motivating for something to work out the first time.</li>
</ul>

<h1>Conclusion</h1>
<ul><li>We scanned 465 search results for the "data science" search term in indeed.com, and we narrowed it down to only 89, which a job seeker can now hand-pick through (even though it's probably best to apply to all of them!).</li></ul>